In [67]:
import pandas as pd
import numpy as np

In [ ]:
stats = pd.read_csv(r'NBA_scraping/data_cleaning/stats_merged.csv')

In [69]:
years = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13',
         '2011-12', '2010-11', '2009-10', '2008-09']

def ewm_year(df, season, span, min_periods, suffix, filename):
    merged = []
    for i in season:
        stats_x = df.loc[df['SEASON_YEAR']== i]        
        stats_x['GAME_DATE'] = pd.to_datetime(stats_x['GAME_DATE'])
        stats_x.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True) 
        stats_x = stats_x.sort_index()
        stats_x =stats_x.groupby('TEAM_ABBREVIATION').ewm(span=span, min_periods=min_periods).mean()
        stats_x = stats_x.shift(1)
        stats_x = stats_x.drop('Unnamed: 0', 1)
        stats_x = stats_x.add_suffix(suffix)
        merged.append(stats_x)
    merged = pd.concat(merged)
    merged.to_csv('%s.csv' %filename)

stats_home = stats
stats_home = stats_home[stats_home.TEAM_ABBREVIATION == stats_home.HOME]
stats_away = stats
stats_away = stats_away[stats_away.TEAM_ABBREVIATION == stats_away.AWAY]


ewm_year(stats, years, 10, 10, ' _ewm10', 'ewm_10')
ewm_year(stats_home, years, 10, 5, '_ewm10ha', 'ewm_10h')
ewm_year(stats_away, years, 10, 5, '_ewm10ha', 'ewm_10a')
ewm_year(stats, years, 5, 5, ' _ewm5', 'ewm_5')
ewm_year(stats_home, years, 5, 5, '_ewm5ha', 'ewm_5h')
ewm_year(stats_away, years, 5, 5, '_ewm5ha', 'ewm_5a') 

<ipython-input-69-f80b1ee370ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_x['GAME_DATE'] = pd.to_datetime(stats_x['GAME_DATE'])


In [70]:
ewm10 = pd.read_csv('ewm_10.csv')
ewm10h = pd.read_csv('ewm_10h.csv')
ewm10a = pd.read_csv('ewm_10a.csv')
ewm5 = pd.read_csv('ewm_5.csv')
ewm5h = pd.read_csv('ewm_5h.csv')
ewm5a = pd.read_csv('ewm_5a.csv')

In [71]:
stats_col = ewm10.reset_index()
stats_home = stats_col
stats_home = stats_home[stats_home.TEAM_ABBREVIATION == stats_home.HOME]
stats_away = stats_col
stats_away = stats_away[stats_away.TEAM_ABBREVIATION == stats_away.AWAY]

In [72]:
stats_home.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_away.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_home = stats_home.drop('TEAM_ABBREVIATION', 1)
stats_away = stats_away.drop('TEAM_ABBREVIATION', 1)

# Subtract stats_home - stats_away if they have the same index
stats_sub = stats_home - stats_away
stats_sub = stats_sub.drop('index', 1)
stats_sub.to_csv('ewm10_diff.csv')

In [73]:
stats_col = ewm5.reset_index()
stats_home = stats_col
stats_home = stats_home[stats_home.TEAM_ABBREVIATION == stats_home.HOME]
stats_away = stats_col
stats_away = stats_away[stats_away.TEAM_ABBREVIATION == stats_away.AWAY]

In [74]:
stats_home.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_away.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_home = stats_home.drop('TEAM_ABBREVIATION', 1)
stats_away = stats_away.drop('TEAM_ABBREVIATION', 1)

# Subtract stats_home - stats_away if they have the same index
stats_sub = stats_home - stats_away
stats_sub = stats_sub.drop('index', 1)
stats_sub.to_csv('ewm5_diff.csv')

In [75]:
stats_10h = ewm10h.reset_index()
stats_home = stats_10h

stats_10a = ewm10a.reset_index()
stats_away = stats_10a

stats_home.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_away.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_home = stats_home.drop('TEAM_ABBREVIATION', 1)
stats_away = stats_away.drop('TEAM_ABBREVIATION', 1)

# Subtract stats_home - stats_away if they have the same index
stats_sub = stats_home - stats_away

stats_sub = stats_sub.drop('index', 1)
stats_sub.to_csv('ewm10_diff_ha.csv')

In [76]:
stats_5h = ewm5h.reset_index()
stats_home = stats_5h

stats_5a = ewm5a.reset_index()
stats_away = stats_5a

stats_home.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_away.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_home = stats_home.drop('TEAM_ABBREVIATION', 1)
stats_away = stats_away.drop('TEAM_ABBREVIATION', 1)

# Subtract stats_home - stats_away if they have the same index
stats_sub = stats_home - stats_away

stats_sub = stats_sub.drop('index', 1)
stats_sub.to_csv('ewm5_diff_ha.csv')

In [78]:
ewm10_diff = pd.read_csv('ewm10_diff.csv')
ewm10_diff_ha = pd.read_csv('ewm10_diff_ha.csv')
ewm5_diff = pd.read_csv('ewm5_diff.csv')
ewm5_diff_ha = pd.read_csv('ewm5_diff_ha.csv')

all_data = ewm10_diff.merge(ewm10_diff_ha).merge(ewm5_diff).merge(ewm5_diff_ha)
all_data.to_csv('stats_ewm_final.csv')